# 🐳 BYOL (Bring Your Own Library) Script Mode（修正版）

このノートブックでは、独自のライブラリを使用したSageMakerスクリプトモードを体験します。

## 🎯 学習目標
- カスタムライブラリの作成と使用
- SageMaker環境でのBYOL実装
- 独自アルゴリズムの実装と実行

## ⏱️ 実行時間目安
- **環境準備**: 3-5分
- **カスタムライブラリテスト**: 5分
- **トレーニング実行**: 10-15分
- **合計**: 約20-25分

## 📝 注意事項
SageMaker Studio環境では、Dockerイメージの構築に制限があります。
このノートブックでは、代替手法を使用してBYOLの概念を学習します。

## 1. 環境設定

In [ ]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import os
import sys
import json
import time
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn

# SageMaker設定
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()
account_id = boto3.client('sts').get_caller_identity()['Account']

print(f"SageMaker role: {role}")
print(f"Region: {region}")
print(f"S3 bucket: {bucket}")
print(f"Account ID: {account_id}")
print(f"Current directory: {os.getcwd()}")
print("\n🎓 BYOL Script Mode講義を開始します")
print("📝 注意: SageMaker Studio環境に最適化された方法を使用します")

## 2. カスタムライブラリの確認と準備

In [ ]:
# BYOLディレクトリの確認
byol_dir = './byol_docker'
if os.path.exists(byol_dir):
    print(f"✅ BYOLディレクトリが存在します: {byol_dir}")
    
    # ファイル一覧表示
    print("\nBYOLファイル構成:")
    for root, dirs, files in os.walk(byol_dir):
        level = root.replace(byol_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            if not file.startswith('.'):
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                print(f"{subindent}{file} ({file_size} bytes)")
else:
    print(f"❌ BYOLディレクトリが見つかりません: {byol_dir}")
    print("\n💡 解決方法:")
    print("1. GitHubから正しくクローンされているか確認")
    print("2. byol_dockerフォルダが存在するか確認")
    raise FileNotFoundError(f"{byol_dir} が見つかりません")

## 3. カスタムライブラリのローカルテスト

### 📚 学習ポイント
- カスタムライブラリの動作確認
- 独自アルゴリズムの理解
- BYOL概念の実践

In [ ]:
# カスタムライブラリをPythonパスに追加
custom_lib_path = os.path.join(byol_dir, 'custom_ml_lib')
if custom_lib_path not in sys.path:
    sys.path.insert(0, custom_lib_path)

print(f"📚 カスタムライブラリパスを追加: {custom_lib_path}")

try:
    # カスタムライブラリをインポート
    from custom_ml_lib.custom_classifier import CustomEnsembleClassifier
    from custom_ml_lib.custom_preprocessor import CustomPreprocessor
    from custom_ml_lib import utils as custom_utils  # 関数としてインポート
    
    print("✅ カスタムライブラリのインポート成功")
    
    # カスタムクラスの動作確認
    print("\n🔍 カスタムライブラリの動作確認:")
    
    # カスタム分類器のテスト
    custom_classifier = CustomEnsembleClassifier(
        use_rf=True, 
        use_gb=True, 
        use_lr=True
    )
    print(f"  📊 カスタム分類器: {type(custom_classifier).__name__}")
    print(f"  🔧 使用モデル数: {len(custom_classifier.models) if hasattr(custom_classifier, 'models') else 'N/A'}")
    
    # カスタム前処理器のテスト
    custom_preprocessor = CustomPreprocessor()
    print(f"  🔄 カスタム前処理器: {type(custom_preprocessor).__name__}")
    
    # カスタムユーティリティのテスト
    print(f"  🛠️ カスタムユーティリティ: {len([func for func in dir(custom_utils) if not func.startswith('_')])}個の関数が利用可能")
    
    print("\n🎉 カスタムライブラリの動作確認完了!")
    
except ImportError as e:
    print(f"❌ カスタムライブラリのインポートに失敗: {str(e)}")
    print("\n🔍 トラブルシューティング:")
    print("1. custom_ml_libフォルダが存在するか確認")
    print("2. __init__.pyファイルが存在するか確認")
    print("3. Pythonファイルの構文エラーがないか確認")
    
    # エラーでも続行
    print("\n⚠️ カスタムライブラリなしで続行します")

## 4. S3データパスの確認

In [ ]:
# S3データパス情報を読み込み
try:
    with open('s3_data_paths.json', 'r') as f:
        s3_paths = json.load(f)
    print("✅ S3データパス情報を読み込み完了")
    for data_type, path in s3_paths.items():
        print(f"  📊 {data_type}: {path}")
except FileNotFoundError:
    print("❌ S3パス情報が見つかりません")
    print("🔄 先に 01_data_management.ipynb を実行してください")
    raise

## 5. BYOL Training Script の準備

### 📚 学習ポイント
- カスタムライブラリを使用した訓練スクリプト
- SageMaker環境での依存関係管理
- 独自アルゴリズムの実装

In [ ]:
# BYOL訓練スクリプトの確認
byol_script_path = os.path.join(byol_dir, 'train.py')
print(f"🔍 BYOL訓練スクリプトを確認中: {byol_script_path}")
if os.path.exists(byol_script_path):
    print(f"✅ BYOL訓練スクリプトが存在します: {byol_script_path}")
    
    # ファイルサイズを確認
    file_size = os.path.getsize(byol_script_path)
    print(f"  📄 ファイルサイズ: {file_size:,} bytes")
    
    # スクリプトの最初の数行を表示
    print("\n📝 スクリプトの内容（最初の10行）:")
    with open(byol_script_path, 'r') as f:
        lines = f.readlines()[:10]
        for i, line in enumerate(lines, 1):
            print(f"  {i:2d}: {line.rstrip()}")
    
    if len(lines) == 10:
        print("     ... (続きあり)")
else:
    print(f"❌ BYOL訓練スクリプトが見つかりません: {byol_script_path}")
    raise FileNotFoundError(f"{byol_script_path} が見つかりません")

# requirements.txtの確認
requirements_path = os.path.join(byol_dir, 'requirements.txt')
if os.path.exists(requirements_path):
    print(f"\n✅ requirements.txtが存在します: {requirements_path}")
    
    # 依存関係を表示
    print("\n📦 依存関係:")
    with open(requirements_path, 'r') as f:
        requirements = f.read().strip().split('\n')
        for req in requirements:
            if req.strip():
                print(f"  - {req.strip()}")
else:
    print(f"\n⚠️ requirements.txtが見つかりません: {requirements_path}")

## 6. SageMaker環境でのBYOL実行

### 📚 学習ポイント
- SageMaker Studio環境でのBYOL実装
- カスタムライブラリの依存関係管理
- 代替手法による高度なカスタマイゼーション

In [ ]:
print("🚀 BYOL Training Jobs実行開始...")
print("📝 注意: SageMaker Studio環境に最適化された方法を使用")

# BYOL用のEstimator設定（修正版）
byol_estimator = SKLearn(
    entry_point='train.py',  # byol_docker内のスクリプト
    source_dir=byol_dir,  # カスタムライブラリを含むディレクトリ
    framework_version='1.0-1',
    py_version='py3',
    instance_type='ml.m5.large',
    instance_count=1,
    role=role,
    hyperparameters={
        'use-custom-ensemble': 'true',
        'ensemble-weights': '0.4,0.4,0.2',  # RF, GB, LR の重み
        'custom-preprocessing': 'true',
        'n-jobs': -1
    },
    base_job_name='byol-lecture',
    max_run=2400  # 最大40分
)

print("✅ BYOL Estimator設定完了")
print(f"  📄 エントリーポイント: train.py (相対パス)")
print(f"  📁 ソースディレクトリ: {byol_dir}")
print(f"  💻 インスタンスタイプ: ml.m5.large")
print(f"  🎛️ カスタムハイパーパラメータ:")
print(f"    - カスタムアンサンブル: 有効")
print(f"    - アンサンブル重み: RF(40%), GB(40%), LR(20%)")
print(f"    - カスタム前処理: 有効")

# コスト情報
hourly_cost = 0.115  # ml.m5.large の時間単価
estimated_time_hours = 15 / 60  # 予想実行時間（15分）
estimated_cost = hourly_cost * estimated_time_hours

print(f"\n💰 コスト情報:")
print(f"  💵 時間単価: ${hourly_cost}/時間")
print(f"  ⏱️ 予想実行時間: 約15分")
print(f"  💸 予想コスト: ${estimated_cost:.4f}")

## 7. BYOL Training Jobsの実行

In [ ]:
print("🎯 BYOL Training Jobs実行中...")
print(f"⏱️ 予想実行時間: 12-18分")
print(f"📊 使用データ: S3に保存された講義用データセット")
print(f"🔬 カスタムライブラリ: 独自アンサンブル分類器")

byol_start = time.time()

try:
    # BYOL Training Jobsを実行
    byol_estimator.fit({
        'train': s3_paths['train'],
        'validation': s3_paths['validation'],
        'test': s3_paths['test']
    })
    
    byol_total_time = time.time() - byol_start
    
    print(f"\n🎉 BYOL Training Jobs完了!")
    print(f"⏱️ 実際の実行時間: {byol_total_time:.1f}秒 ({byol_total_time/60:.1f}分)")
    
    # 実行成功フラグ
    byol_success = True
    
except Exception as e:
    print(f"\n❌ BYOL Training Jobsでエラーが発生: {str(e)}")
    print("\n🔍 一般的な原因:")
    print("1. カスタムライブラリの依存関係の問題")
    print("2. 訓練スクリプトの構文エラー")
    print("3. SageMaker環境での権限問題")
    print("4. リソース不足")
    
    # エラーでも続行
    print("\n⚠️ BYOLデモンストレーションとして続行します")
    byol_success = False
    byol_total_time = time.time() - byol_start

## 8. BYOL結果の分析

In [ ]:
if byol_success:
    # 成功した場合の詳細分析
    training_job_name = byol_estimator.latest_training_job.job_name
    print(f"📋 BYOL Training Job詳細:")
    print(f"  🏷️ ジョブ名: {training_job_name}")
    
    # ジョブの実行統計
    training_job_description = sagemaker_session.describe_training_job(training_job_name)
    
    print(f"\n📊 実行統計:")
    print(f"  ✅ ジョブ状態: {training_job_description['TrainingJobStatus']}")
    print(f"  💻 インスタンスタイプ: {training_job_description['ResourceConfig']['InstanceType']}")
    
    # コスト計算
    billable_seconds = training_job_description.get('BillableTimeInSeconds', 0)
    actual_cost = (billable_seconds / 3600) * hourly_cost
    
    print(f"\n💰 コスト詳細:")
    print(f"  ⏱️ 課金時間: {billable_seconds}秒 ({billable_seconds/60:.1f}分)")
    print(f"  💵 実際のコスト: ${actual_cost:.4f}")
    
    # モデルアーティファクト
    model_artifacts = byol_estimator.model_data
    print(f"\n📦 出力情報:")
    print(f"  📄 モデルアーティファクト: {model_artifacts}")
    print(f"  📊 CloudWatchログ: /aws/sagemaker/TrainingJobs/{training_job_name}")
    
else:
    # エラーの場合のデモンストレーション
    print("📚 BYOLの概念デモンストレーション:")
    print("\n🎯 BYOLで実現できること:")
    
    concepts = [
        "独自のアルゴリズム実装",
        "企業秘密の技術活用",
        "特殊なライブラリの統合",
        "カスタム前処理パイプライン",
        "業界特化のモデル開発",
        "完全な環境制御"
    ]
    
    for i, concept in enumerate(concepts, 1):
        print(f"  {i}. ✨ {concept}")
    
    print(f"\n⏱️ デモ実行時間: {byol_total_time:.1f}秒")

## 9. 03・05・06の比較まとめ

In [ ]:
print("📊 SageMaker機械学習手法の比較まとめ")
print("=" * 60)

comparison_data = {
    "手法": ["03 Training Jobs", "05 Built-in Algorithms", "06 BYOL"],
    "カスタマイズ度": ["中程度", "低い", "最高"],
    "実装難易度": ["中程度", "低い", "高い"],
    "実行速度": ["標準", "高速", "カスタム依存"],
    "適用場面": ["一般的なML", "標準問題", "特殊要件"]
}

# 表形式で表示
import pandas as pd
comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

print("\n🎯 使い分けの指針:")
print("\n📈 学習・実験段階:")
print("  → 03 Training Jobs でプロトタイプ開発")

print("\n🚀 本番・効率重視:")
print("  → 05 Built-in Algorithms で高速化")

print("\n🔬 差別化・特殊要件:")
print("  → 06 BYOL で独自技術実装")

print("\n💡 実際のプロジェクトでは:")
print("  1. 03で概念実証（PoC）")
print("  2. 05で標準業務の効率化")
print("  3. 06で競合優位性の確立")

print(f"\n🎉 BYOL講義完了!")
print(f"📚 学習内容:")
learning_outcomes = [
    "カスタムライブラリの概念理解",
    "SageMaker環境での制約と対処法",
    "BYOL実装の基本パターン",
    "企業レベルのML開発手法",
    "3つの手法の適切な使い分け"
]

for i, outcome in enumerate(learning_outcomes, 1):
    print(f"  {i}. ✅ {outcome}")

print("\n🚀 次のステップ: 実際のプロジェクトでの活用!")